In [ ]:
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers
!pip -q install gradio
!pip -q install torch
!pip -q install openai

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

openai.api_key = "Paste your openai api key here"

In [ ]:
#dependencies
import gradio as gr
import openai
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# response to audio
def tts(input_text):
    inputs = processor(text=input_text, return_tensors="pt")
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    audio_numpy = speech.numpy()
    sample_rate = 16000

    return(sample_rate,audio_numpy)

# prompt response
def openai_chat(prompt):
    prompt = prompt + "Respond in less than 50 words"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=1,
        max_tokens=250,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    audio_response = tts(response.choices[0].message.content)

    return(response.choices[0].message.content,audio_response)

#Gradio Interface
iface = gr.Interface(
    fn=openai_chat,
    inputs=gr.Textbox("Hi how are you", label="Converse with ChatGPT"),
    outputs=['text',gr.Audio(label="Audio Player")],
    live=False,
    title="converse.ai",
    theme="light",
    layout="vertical",
    button=gr.Button("Submit")
)

iface.launch(share=True)